In [1]:
import pandas as pd

In [2]:
import re
from stop_words import get_stop_words
stopwords = get_stop_words('english') 

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

In [2]:
from collections import Counter
import numpy as np

In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [3]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

c:\programdata\anaconda3\envs\mlenv\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [4]:
from tqdm import tqdm_notebook

In [5]:
import pickle

In [6]:
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve

In [8]:
abstracts = pd.read_csv('papers_with_abstract.csv')

In [9]:
abstracts.head()

,id,text,text_len
0,E03-1062,This paper describes the NECA MNLG;\r\r\na ful...,15690
1,E06-1001,I propose a uniform approach to the elim-\r\r\...,28328
2,E06-1002,We present a new method for detecting and\r\r\...,33424
3,E06-1003,We present a weakly supervised approach\r\r\nt...,31859
4,E06-1004,In this paper we study a set of prob-\r\r\nlem...,26466


In [10]:
get_abstract = lambda x: x.split('\nAbstract\n')[-1].split('\n1 ')[0]

In [11]:
abstracts.iloc[0].text.split('\nAbstract\n')[-1].split('\n1')[0]

'This paper describes the NECA MNLG;\r\r\na fully implemented Multimodal Natu-\r\r\nral Language Generation module. The\r\r\nMNLG is deployed as part of the NECA\r\r\nsystem which generates dialogues be-\r\r\ntween animated agents. The genera-\r\r\ntion module supports the seamless inte-\r\r\ngration of full grammar rules, templates\r\r\nand canned text. The generator takes in-\r\r\nput which allows for the specification of\r\r\nsyntactic, semantic and pragmatic con-\r\r\nstraints on the output.\r\r'

In [12]:
abstracts['abstract'] = abstracts['text'].apply(get_abstract)

In [13]:
def clean(text, stopwords):
    text = text.replace('\r','').replace('-\n', '').replace('\n', ' ').lower()
    tokens = re.findall(r'[a-z]+', text)
    return ' '.join([token for token in tokens if token not in stopwords])

In [14]:
clean(abstracts.iloc[0]['abstract'], stopwords)

'paper describes neca mnlg fully implemented multimodal natural language generation module mnlg deployed part neca system generates dialogues animated agents generation module supports seamless integration full grammar rules templates canned text generator takes input allows specification syntactic semantic pragmatic constraints output'

In [15]:
abstracts['abstract'] = [clean(abstr, stopwords) for abstr in tqdm_notebook(abstracts['abstract'])]

In [13]:
abstracts.iloc[1000].abstract

'limited coverage lexical semantic resources significant problem nlp systems can alleviated automatically classifying unknown words supersense tagging assigns unknown nouns one broad semantic categories used lexicographers organise manual insertion wordnet ciaramita johnson present tagger uses synonym set glosses annotated training examples describe unsupervised approach based vector space similarity require annotated examples significantly outperforms tagger also demonstrate use extremely large shallow parsed corpus calculating vector space semantic similarity'

In [11]:
with open('./asymproj_edge_dnn-master/datasets/aan_graph/index.pkl', 'rb') as f:
    index_graph = pickle.load(f)

In [13]:
index_graph['index'][abstracts.iloc[1000].id] #for node in aan_links_connected['citing']

In [18]:
def map_node_id(node, map_dict):
    node_id = None
    try:
        node_id = map_dict[node]
    except KeyError:
        pass
    return node_id

In [19]:
node_ids = [map_node_id(node, index_graph['index']) for node in tqdm_notebook(abstracts['id'])]

In [20]:
map_node_id(abstracts.iloc[0].id, index_graph['index'])

2517

In [21]:
abstracts['node_id'] = node_ids

In [22]:
abstracts.shape

(16268, 5)

In [23]:
abstracts

,id,text,text_len,abstract,node_id
0,E03-1062,This paper describes the NECA MNLG;\r\r\na ful...,15690,paper describes neca mnlg fully implemented mu...,2517.0
1,E06-1001,I propose a uniform approach to the elim-\r\r\...,28328,propose uniform approach elimination redundanc...,2518.0
2,E06-1002,We present a new method for detecting and\r\r\...,33424,present new method detecting disambiguating na...,2519.0
3,E06-1003,We present a weakly supervised approach\r\r\nt...,31859,present weakly supervised approach automatic o...,2520.0
4,E06-1004,In this paper we study a set of prob-\r\r\nlem...,26466,paper study set problems considerable importan...,2521.0
5,E06-1005,This paper describes a novel method for\r\r\nc...,28598,paper describes novel method computing consens...,2522.0
6,E06-1006,We propose a backoff model for phrase-\r\r\nba...,31962,propose backoff model phrasebased machine tran...,2523.0
7,E06-1007,We present an implemented machine\r\r\nlearnin...,35869,present implemented machine learning system au...,2524.0
8,E06-1008,"In this paper, we explore statistical lan-\r\r...",33343,paper explore statistical language modelling s...,2525.0
9,E06-1009,To tackle the problem of presenting a\r\r\nlar...,35245,tackle problem presenting large number options...,2526.0


In [24]:
input_text = abstracts[abstracts['node_id'].notnull()][['id','node_id','abstract']].sort_values(by = 'node_id')

In [26]:
input_text['node_id'] = input_text['node_id'].astype(int)

In [27]:
input_text

,id,node_id,abstract
8090,A00-1002,0,using examples transfer based mt system czech ...
8091,A00-1005,1,paper describes system provides customer servi...
8092,A00-1006,2,paper proposes way improve translation quality...
8093,A00-1008,3,paper describes application ape atlas planning...
8094,A00-1009,4,paper describe implemented framework developin...
8095,A00-1010,5,describe talk n travel spoken dialogue languag...
8096,A00-1011,6,paper reports large scale end toend relation e...
8098,A00-1014,7,paper describes mimic adaptive mixed initiativ...
8099,A00-1018,8,past decade lot work computational linguistics...
8100,A00-1021,9,paper describe system rank suspected answers n...


In [52]:
input_text[['abstract']].to_csv('aan_text.txt', header=None, index=False)

In [ ]:
# %%time
# !pctadw --input_text aan_text.txt --input_edges aan_connected.edgelist --split_sample_size 10000 --model_name PCTADW-2 --output pctadw.embedding 

In [28]:
abstracts.head()

,id,text,text_len,abstract,node_id
0,E03-1062,This paper describes the NECA MNLG;\r\r\na ful...,15690,paper describes neca mnlg fully implemented mu...,2517.0
1,E06-1001,I propose a uniform approach to the elim-\r\r\...,28328,propose uniform approach elimination redundanc...,2518.0
2,E06-1002,We present a new method for detecting and\r\r\...,33424,present new method detecting disambiguating na...,2519.0
3,E06-1003,We present a weakly supervised approach\r\r\nt...,31859,present weakly supervised approach automatic o...,2520.0
4,E06-1004,In this paper we study a set of prob-\r\r\nlem...,26466,paper study set problems considerable importan...,2521.0


In [25]:
import numpy as np

In [135]:
root_embed = 'D:/wiki.en.vec'

In [136]:
words = []

embeddings_index = {}
fo = open(root_embed, encoding = 'utf-8')
for line in tqdm_notebook(fo):
    values = line.split()
    if len(values) == 301:
        word = values[0]
        words.append(word)
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
fo.close()

In [138]:
def mean_text_embedding(text, model, dim):
    text = text.split()
    words = Counter(text)
    total = len(text)
    vectors = np.zeros((len(words), dim))
    
    for i,word in enumerate(words):
        try:
            v = model[word]
            vectors[i] = v*(words[word]/total) 
        except (KeyError, ValueError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector

In [139]:
dim = 300
X_mean_text_emb = np.zeros((len(input_text['abstract']), dim))

for i, text in enumerate(input_text['abstract'].values):
    X_mean_text_emb[i] = mean_text_embedding(text, embeddings_index, dim)

In [140]:
X_mean_text_emb.shape

(13506, 300)

In [32]:
X_mean_text_emb = np.load('X_mean_text_emb_13506.npy')

In [33]:
X_mean_text_emb.shape

(13506, 300)

In [141]:
# np.save('X_mean_text_emb.npy', X_mean_text_emb)
np.save('X_mean_text_emb_13506.npy', X_mean_text_emb)

In [221]:
X_mean_text_emb = np.load('X_mean_text_emb_13506.npy')

In [222]:
X_mean_text_emb.shape

(13506, 300)

In [223]:
emb_df = pd.DataFrame(data = X_mean_text_emb)

In [227]:
emb_df.to_csv('mean_emb_features.txt', sep = ' ', header = None)

In [6]:
tfidf = TfidfVectorizer(min_df=2, max_df=0.4, max_features = 20000)
X_tfidf = tfidf.fit_transform(input_text['abstract'])

In [16]:
type(X_tfidf)

scipy.sparse.csr.csr_matrix

In [37]:
stemmer.stem('baseline')

'baselin'

In [41]:
wordnet_lemmatizer.lemmatize('arises')

'arises'

In [142]:
idf_dict = dict([(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

In [143]:
def tfidf_text_embedding(text, model, dim, idf):
    text = text.split()
    
    words = Counter(text)
    total = len(text)
    vectors = np.zeros((len(words), dim))
    
    for i,word in enumerate(words):
        try:
            v = model[word]
            vectors[i] = v*(words[word]/total)*idf[word] 
        except (KeyError, ValueError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector

In [144]:
dim = 300
X_mean_text_emb = np.zeros((len(input_text['abstract']), dim))

for i, text in enumerate(input_text['abstract'].values):
    X_mean_text_emb[i] = tfidf_text_embedding(text, embeddings_index, dim, idf_dict)

In [145]:
# np.save('X_tfidf_text_emb.npy', X_mean_text_emb)
np.save('X_tfidf_text_emb_13506.npy', X_mean_text_emb)

In [167]:
X_tfidf_weighted_text_emb = np.load('X_tfidf_text_emb_13506.npy')

In [79]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(input_text['abstract'])]

In [81]:
d2v = Doc2Vec(documents, vector_size=300, window=4, min_count=2)

In [83]:
type(d2v)

gensim.models.doc2vec.Doc2Vec

In [84]:
d2v.save('d2v_embed_13506_300')
# model = Doc2Vec.load(fname)

In [8]:
d2v = Doc2Vec.load('d2v_embed_13506')

In [ ]:
### TFIDF

In [48]:
tfidf = TfidfVectorizer(min_df=2, max_df=0.4, max_features = 20000)
X_tfidf = tfidf.fit_transform(input_text['abstract'])

In [49]:
X_tfidf.shape

(13506, 20000)

In [9]:
test_x_y = pd.read_csv('test_x_y_05.csv')

In [10]:
test_x_y.shape

(22900, 3)

In [50]:
X_tfidf.shape

(13506, 20000)

In [51]:
X_tfidf[index_graph['index'][test_x_y.iloc[0]['citing']]].shape

(1, 20000)

In [82]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [83]:
sigmoid(cosine_similarity(X_tfidf[index_graph['index'][test_x_y.iloc[3]['citing']]], X_tfidf[index_graph['index'][test_x_y.iloc[3]['cited']]])[0][0])

0.5333610014967234

In [89]:
sigmoid(cosine_similarity(X_tfidf[index_graph['index'][test_x_y.iloc[0]['citing']]], X_tfidf[index_graph['index'][test_x_y.iloc[0]['cited']]])[0][0])

0.5011053233025877

In [103]:
cosine_similarity(X_tfidf[index_graph['index'][test_x_y.iloc[0]['citing']]], X_tfidf[index_graph['index'][test_x_y.iloc[0]['cited']]])[0][0]

0.0044213004125975165

In [44]:
# X_tfidf = np.load('X_tfidf_text_emb_13506.npy')

In [52]:
X_tfidf

<13506x20000 sparse matrix of type '<class 'numpy.float64'>'
	with 977047 stored elements in Compressed Sparse Row format>

In [53]:
tfidf_matrix = X_tfidf
link_to_matrix_id_dict = index_graph['index']
cos_sim = lambda x: cosine_similarity(tfidf_matrix[link_to_matrix_id_dict[x['citing']]],
                                                  tfidf_matrix[link_to_matrix_id_dict[x['cited']]])[0][0]

In [54]:
X_tfidf[1080]

<1x20000 sparse matrix of type '<class 'numpy.float64'>'
	with 66 stored elements in Compressed Sparse Row format>

In [55]:
tfidf_matrix.shape

(13506, 20000)

In [56]:
from scipy import sparse

sparse.save_npz("X_tfidf_13506.npz", tfidf_matrix)
# tfidf_matrix = sparse.load_npz("X_tfidf_13506.npz")

In [57]:
tfidf_matrix[link_to_matrix_id_dict[test_x_y.iloc[0]['citing']]]

<1x20000 sparse matrix of type '<class 'numpy.float64'>'
	with 66 stored elements in Compressed Sparse Row format>

In [24]:
test_x_y.head()

,citing,cited,class,sim_d2v
0,C14-1216,W07-1019,0,0.454362
1,W13-2002,D08-1027,0,0.547518
2,D13-1041,W14-1304,0,0.604749
3,P14-2095,P13-2017,1,0.260585
4,C14-1089,W04-3217,0,0.505673


In [58]:
cos_sim_links = test_x_y.apply(cos_sim, axis = 1)

In [59]:
max(cos_sim_links)

1.0000000000000002

In [60]:
np.mean(cos_sim_links)

0.074857714621801

In [61]:
test_x_y.shape

(22900, 4)

In [134]:
len([i for i, s in enumerate(cos_sim_links) if s > 0.05])

10449

In [116]:
test_x_y.iloc[21004]

citing    W13-3102
cited     W13-3102
class            1
Name: 21004, dtype: object

In [126]:
from sklearn.metrics import roc_auc_score

In [127]:
roc_auc_score(test_x_y['class'], cos_sim_links)

0.8668837207528461

In [131]:
log_loss(test_x_y['class'], cos_sim_links)

1.3964500520303407

In [ ]:
## FastTexxt + CosSim

In [1]:
# emb_matrix[link_to_matrix_id_dict[test_x_y.iloc[0]['citing']]]

In [161]:
cosine_similarity([emb_matrix[link_to_matrix_id_dict[test_x_y.iloc[0]['citing']]]],\
                  [emb_matrix[link_to_matrix_id_dict[test_x_y.iloc[0]['cited']]]])

array([[0.78537427]])

In [62]:
X_mean_text_emb = np.load('X_mean_text_emb_13506.npy')

In [64]:
emb_matrix = X_mean_text_emb
link_to_matrix_id_dict = index_graph['index']
cos_sim_emb = lambda x: cosine_similarity([emb_matrix[link_to_matrix_id_dict[x['citing']]]],
                                                  [emb_matrix[link_to_matrix_id_dict[x['cited']]]])[0][0]

In [65]:
cos_sim_links_ft = test_x_y.apply(cos_sim_emb, axis = 1)

In [66]:
roc_auc_score(test_x_y['class'], cos_sim_links_ft)

0.7376886710779733

In [67]:
log_loss(test_x_y['class'], cos_sim_links_ft)

1.146666899129114

In [68]:
X_tfidf_weighted_text_emb = np.load('X_tfidf_text_emb_13506.npy')

In [69]:
emb_matrix = X_tfidf_weighted_text_emb
link_to_matrix_id_dict = index_graph['index']
cos_sim_emb = lambda x: cosine_similarity([emb_matrix[link_to_matrix_id_dict[x['citing']]]],
                                                  [emb_matrix[link_to_matrix_id_dict[x['cited']]]])[0][0]

In [70]:
cos_sim_links_ft_weighted = test_x_y.apply(cos_sim_emb, axis = 1)

In [71]:
roc_auc_score(test_x_y['class'], cos_sim_links_ft_weighted)

0.7199453557331096

In [175]:
log_loss(test_x_y['class'], cos_sim_links_ft_weighted)

1.0660873099570318

In [85]:
emb_matrix = d2v
link_to_matrix_id_dict = index_graph['index']
cos_sim_emb = lambda x: cosine_similarity([emb_matrix[link_to_matrix_id_dict[x['citing']]]],
                                                  [emb_matrix[link_to_matrix_id_dict[x['cited']]]])[0][0]

In [86]:
cos_sim_links_d2v = test_x_y.apply(cos_sim_emb, axis = 1)

In [87]:
roc_auc_score(test_x_y['class'], cos_sim_links_d2v)

0.589450891478042

In [88]:
log_loss(test_x_y['class'], cos_sim_links_d2v)

0.7336357228573841

In [183]:
fpr, tpr, thresholds = roc_curve(test_x_y['class'], cos_sim_links_ft_weighted)

In [186]:
thresholds

array([1.        , 0.95637849, 0.95629264, ..., 0.6291629 , 0.62070531,
       0.54346961])

In [185]:
fpr 

array([0.00000000e+00, 0.00000000e+00, 8.73362445e-05, ...,
       9.99825328e-01, 9.99825328e-01, 1.00000000e+00])

In [187]:
tpr 

array([1.74672489e-04, 6.63755459e-03, 6.63755459e-03, ...,
       9.99912664e-01, 1.00000000e+00, 1.00000000e+00])

In [188]:
threshold_list = [0.5, 0.3, 0.1, 0.01]

In [197]:
weighted_labels = cos_sim_links_ft_weighted.copy()

In [191]:
weighted_labels[weighted_labels < 0.5] = 0

In [193]:
weighted_labels[weighted_labels > 0.1] = 1

In [201]:
min(weighted_labels)

0.5434696145883092

In [76]:
test_x_y['sim_tfidf'] = cos_sim_links
test_x_y['sim_ft_mean'] = cos_sim_links_ft
test_x_y['sim_ft_weighted'] = cos_sim_links_ft_weighted


In [90]:
test_x_y = pd.read_csv('test_links_class_similarities_2505.csv')

In [92]:
test_x_y['sim_d2v'] = cos_sim_links_d2v

In [94]:
test_x_y.to_csv('test_links_class_similarities_2505.csv')

In [206]:
cos_sim_links_ft_weighted[3]

0.8544602281675031

In [211]:
test_x_y[test_x_y['class'] == 1].min()

citing             C14-1002
cited              C02-2020
class                     1
sim_tfidf                 0
sim_ft_mean        0.731348
sim_ft_weighted    0.620705
dtype: object

In [212]:
test_x_y[test_x_y['class'] == 0].min()

citing             C14-1002
cited              A00-1009
class                     0
sim_tfidf                 0
sim_ft_mean        0.596118
sim_ft_weighted     0.54347
dtype: object

In [213]:
test_x_y[test_x_y['class'] == 1].mean()

class              1.000000
sim_tfidf          0.120710
sim_ft_mean        0.910623
sim_ft_weighted    0.890859
dtype: float64

In [214]:
test_x_y[test_x_y['class'] == 0].mean()

class              0.000000
sim_tfidf          0.029006
sim_ft_mean        0.884889
sim_ft_weighted    0.861149
dtype: float64

In [216]:
test_x_y[test_x_y['class'] == 1].max()

citing             W14-6206
cited              X98-1026
class                     1
sim_tfidf                 1
sim_ft_mean               1
sim_ft_weighted           1
dtype: object

In [219]:
input_text.to_csv('graph_id_clean_text.csv', index = False)

In [20]:
input_text = pd.read_csv('graph_id_clean_text.csv')

In [215]:
test_x_y[test_x_y['class'] == 0].max()

citing             W14-6205
cited              X98-1028
class                     0
sim_tfidf          0.312674
sim_ft_mean        0.960478
sim_ft_weighted    0.956293
dtype: object

In [78]:
test_x_y.to_csv('test_links_class_similarities.csv', index=False)

In [ ]:
### WMD